In [478]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import sys
!{sys.executable} -m pip install xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier,RandomForestClassifier,StackingClassifier


In [479]:
X_train = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")
y_train = X_train["Transported"]
X_train.drop(["Transported","Name"],axis = 1,inplace = True)
X_test.drop("Name",axis = 1, inplace = True)

In [480]:
X_train["PassengerId"] = X_train["PassengerId"].astype(str)
X_test["PassengerId"] = X_test["PassengerId"].astype(str)

Id_train = X_train["PassengerId"]
Id_test = X_test["PassengerId"]

In [481]:
X_train["Group"] = X_train["PassengerId"]
X_test["Group"] = X_test["PassengerId"]

In [482]:
X_train["Group"] = X_train["Group"].apply(lambda x : x[-2:])
X_train["Id"]= X_train["PassengerId"].apply(lambda x: x[:-3])


X_train["Group"] = X_train["Group"].apply(lambda x : x[-2:])
X_test["Id"]= X_test["PassengerId"].apply(lambda x: x[:-3])

In [483]:
X_train["Group"] = X_train["Group"].astype(int)
X_test["Group"] = X_test["Group"].astype(int)


In [484]:




X_train.drop(["Id","PassengerId"],axis = 1,inplace = True)
X_test.drop(["Id","PassengerId"],axis = 1,inplace = True)

In [485]:
X_train["Cabin"] = X_train["Cabin"].astype(str) 
X_test["Cabin"] = X_test["Cabin"].astype(str) 

In [486]:
X_train["Cabin_0"] = X_train["Cabin"].apply(lambda x: x[0])
X_train["Cabin_1"] = X_train["Cabin"].apply(lambda x: x[2:-2])
X_train["Cabin_2"] = X_train["Cabin"].apply(lambda x : x[-1])



X_test["Cabin_0"] = X_test["Cabin"].apply(lambda x: x[0])
X_test["Cabin_1"] = X_test["Cabin"].apply(lambda x: x[2:-2])
X_test["Cabin_2"] = X_test["Cabin"].apply(lambda x : x[-1])

In [487]:
X_train.drop(["Cabin","Cabin_1"],axis = 1, inplace = True)
X_test.drop(["Cabin","Cabin_1"],axis = 1, inplace = True)

In [488]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols,axis = 1)


In [489]:
num_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(num_imputer.fit_transform(num_X_train))
imputed_X_test = pd.DataFrame(num_imputer.transform(num_X_test))
imputed_X_train.columns = num_X_train.columns
imputed_X_test.columns = num_X_test.columns


In [490]:
imputed_X_train["Totalspent"] = imputed_X_train["RoomService"] + imputed_X_train["FoodCourt"] + imputed_X_train["ShoppingMall"] + imputed_X_train["Spa"] + imputed_X_train["VRDeck"]
imputed_X_test["Totalspent"] = imputed_X_test["RoomService"] + imputed_X_test["FoodCourt"] + imputed_X_test["ShoppingMall"] + imputed_X_test["Spa"] + imputed_X_test["VRDeck"]

In [491]:
ob_X_train = X_train[object_cols]
ob_X_test = X_test[object_cols]

In [492]:
OH_encoder = OneHotEncoder(handle_unknown = "ignore",sparse = False)
oh_X_train = pd.DataFrame(OH_encoder.fit_transform(ob_X_train))
oh_X_test = pd.DataFrame(OH_encoder.transform(ob_X_test))
oh_X_train.index = X_train.index
oh_X_test.index = X_test.index

In [493]:
x_train = pd.concat([oh_X_train,imputed_X_train],axis = 1)
x_test = pd.concat([oh_X_test,imputed_X_test],axis = 1)

In [494]:
y_train = np.asarray([1 if i == True else 0 for i in y_train])

In [495]:
clf = XGBClassifier(max_depth = 30,learning_rate = 0.01,n_estimators = 150)
clf.fit(x_train,y_train)
scores = cross_val_score(clf, x_train, y_train, cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())

Mean cross-validation score: 0.79


In [496]:
x_train = x_train.to_numpy()

In [497]:
a.shape

(8693, 34)

In [498]:
models = []
AdaBoostClassifier = AdaBoostClassifier()
BaggingClassifier = BaggingClassifier()
ExtraTreesClassifier = ExtraTreesClassifier()
GradientBoostingClassifier = GradientBoostingClassifier()
HistGradientBoostingClassifier = HistGradientBoostingClassifier()
RandomForestClassifier = RandomForestClassifier()
models.append(AdaBoostClassifier)
models.append(BaggingClassifier)
models.append(ExtraTreesClassifier)
models.append(GradientBoostingClassifier)
models.append(HistGradientBoostingClassifier)
models.append(RandomForestClassifier)
i = 0
for model in models:
    model.fit(x_train,y_train)
    scores = cross_val_score(model, x_train, y_train, cv=10)
    print(f'model index {type(model).__name__}')
    print("Mean cross-validation score: %.2f" % scores.mean())


model index AdaBoostClassifier
Mean cross-validation score: 0.79
model index BaggingClassifier
Mean cross-validation score: 0.78
model index ExtraTreesClassifier
Mean cross-validation score: 0.77
model index GradientBoostingClassifier
Mean cross-validation score: 0.80
model index HistGradientBoostingClassifier
Mean cross-validation score: 0.80
model index RandomForestClassifier
Mean cross-validation score: 0.79


In [499]:
predictions = pd.DataFrame(GradientBoostingClassifier.predict(x_test.to_numpy()))
Id = pd.DataFrame(Id_test)
output = pd.concat([Id,predictions],axis = 1)

In [500]:
output.rename(columns = {"Id": "PassengerId", 0: "Transported"},inplace=True)

In [501]:
def bool(x):
    if x == 1:
        return True
    else:
        return False
output["Transported"]= output["Transported"].apply(bool)

In [502]:
output

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [503]:
output.to_csv("prediction.csv",index = False)